In [9]:
from datetime import datetime
from qiskit.providers.fake_provider import fake_backend
from qiskit.providers.models import BackendProperties, QasmBackendConfiguration

curr_time = str(datetime.now())


In [7]:
def generate_configuration(name, n_qubits, cmap):
    return QasmBackendConfiguration(
            backend_name=name,
            n_qubits=n_qubits,
            basis_gates=["id", "rz", "sx", "x", "cx"],
            simulator=False,
            local=True,
            coupling_map=cmap,
            backend_version="0.0.1",
            gates=[],
            open_pulse=False,
            conditional=False,
            memory=False,
            max_shots=float('inf'),
            max_experiments=float('inf'),
            supported_instructions=["measure", "u3", "setf", "x", "delay", "id", "acquire", "sx", "u2", "u1", "cx", "shiftf", "play", "rz", "reset"],
            multi_meas_enabled=False
        )


In [8]:
def generate_properties(name, n_qubits, cmap, single_errors, double_errors, meas_errors):
    properties = {"backend_version": "0.0.1", "general": [], "last_update_date": now, "backend_name": name}
    
    qubit_properties_template = [
        ['T1', 600, 'µs'],
        ['T2', 600, 'µs'],
        ['frequency', 5, 'GHz'],
        ['readout', 0.05, ''],
        ["prob_meas0_prep1", 0.08, ''],
        ["prob_meas1_prep0", 0.02, ''],
        
    qubit_properties = []
    for i in range(n_qubits):
        qubit_property = []
        for template in qubit_properties_template: 
            qubit_property.append(qubit_property_builder(*template))
        qubit_properties.append(qubit_property)
    

def qubit_property_builder(name, value, unit):
    return {"date": curr_time, "unit": unit, "value": value, "name": name}
    

In [ ]:
class FakeIonq16(fake_backend.FakeBackend):
    """A fake 16 qubit fully connected backend."""

    def __init__():
        super().super().__init__(
            provider=None,
            name='FakeIonq16'            
        )
        self._target = None
        self.sim = None
        
        
    def gen_conf():
        return {
            
        }
        